## Ingesting PDF

In [1]:
%pip install --q unstructured langchain langchain-community
%pip install --q "unstructured[all-docs]" ipywidgets tqdm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from IPython.display import display as Markdown
from tqdm.autonotebook import tqdm as notebook_tqdm

/tmp/ipykernel_197177/3632673901.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm as notebook_tqdm


In [3]:
# Désinstalle torch, torchvision, torchaudio puis réinstalle torch et torchvision (CPU)
!pip uninstall -y torch torchvision torchaudio
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu

Found existing installation: torch 2.7.0
Uninstalling torch-2.7.0:
  Successfully uninstalled torch-2.7.0
Found existing installation: torchvision 0.22.0
Uninstalling torchvision-0.22.0:
  Successfully uninstalled torchvision-0.22.0
Looking in indexes: https://download.pytorch.org/whl/cpu
  Using cached https://download.pytorch.org/whl/cpu/torch-2.7.0%2Bcpu-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (27 kB)
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.22.0%2Bcpu-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (6.1 kB)
Using cached https://download.pytorch.org/whl/cpu/torch-2.7.0%2Bcpu-cp312-cp312-manylinux_2_28_x86_64.whl (175.8 MB)
Using cached https://download.pytorch.org/whl/cpu/torchvision-0.22.0%2Bcpu-cp312-cp312-manylinux_2_28_x86_64.whl (2.0 MB)


In [4]:
import os

local_path = "/home/perfect/Documents/GitHub/ollama_pdf_rag/data/pdfs/sample/1742719496377.pdf"

if os.path.exists(local_path):
    loader = UnstructuredPDFLoader(file_path=local_path)
    data = loader.load()
    print("Chargement réussi, nombre de pages :", len(data))
else:
    print("Fichier non trouvé :", local_path)

Chargement réussi, nombre de pages : 1


In [5]:
# import os
# from langchain_community.document_loaders import UnstructuredPDFLoader

# # local_path = "WEF_The_Global_Cooperation_Barometer_2024.pdf"
# local_path = "data/pdfs/sample/WEF_The_Global_Cooperation_Barometer_2024.pdf"


# if os.path.exists(local_path):
#     loader = UnstructuredPDFLoader(file_path=local_path)
#     data = loader.load()
#     print("Chargement réussi, nombre de pages :", len(data))
# else:
#     print("Fichier non trouvé :", local_path)


In [6]:
# Preview first page
Markdown(data[0].page_content)

"Data\tScience\tfrom\tScratch\n\nThe\t#1\tData\tScience\tGuide\tfor\tEverything A\tData\tScientist\tNeeds\tto\tKnow:\tPython, Linear\tAlgebra,\tStatistics,\tCoding, Applications,\tNeural\tNetworks,\tand Decision\tTrees\n\nSteven\tCooper\n\nTable\tof\tContents\n\nPreface\n\nIntroduction\n\nData\tScience\tand\tits\tImportance\n\nWhat\tis\tit\tExactly?\n\nWhy\tIt\tMatters\n\nWhat\tYou\tNeed\n\nThe\tAdvantages\tto\tData\tScience Data\tScience\tand\tBig\tData\n\nKey\tDifference\tBetween\tData\tScience\tand\tBig\tData\n\nData\tScientists\n\nThe\tProcess\tof\tData\tScience Responsibilities\tof\ta\tData\tScientist Qualifications\tof\tData\tScientists Would\tYou\tBe\ta\tGood\tData\tScientist?\n\nThe\tImportance\tof\tHacking The\tImportance\tof\tCoding\n\nWriting\tProduction-Level\tCode\n\nPython SQL\n\nR\n\nSAS\n\nJava Scala\n\nJulia\n\nHow\tto\tWork\twith\tData\n\nData\tCleaning\tand\tMunging\n\nData\tManipulation\n\nData\tRescaling\n\nPython\n\nInstalling\tPython\n\nPython\tLibraries\tand\tDa

## Vector Embeddings

In [7]:

!ollama list

NAME                                                 ID              SIZE      MODIFIED     
nomic-embed-text:latest                              0a109f422b47    274 MB    21 hours ago    
hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF:latest    042cf58aa32f    807 MB    24 hours ago    
hf.co/CompendiumLabs/bge-base-en-v1.5-gguf:latest    98c4eb4a3287    68 MB     27 hours ago    


In [8]:
# # 1. First clean up any existing ChromaDB installations
# %pip uninstall -y chromadb
# %pip uninstall -y protobuf

# # 2. Install specific versions known to work together
# %pip install -q protobuf==3.20.3
# %pip install -q chromadb==0.4.22  # Using a stable older version
# %pip install -q langchain-ollama

# # 3. Set the environment variable
# import os
# os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"
# Nettoyage et installation des versions compatibles de chromadb, protobuf et langchain-ollama
%pip uninstall -y chromadb
%pip uninstall -y protobuf

%pip install -q protobuf==3.20.3
%pip install -q chromadb==0.4.22
%pip install -q langchain-ollama

import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

Note: you may need to restart the kernel to use updated packages.
Found existing installation: protobuf 6.31.1
Uninstalling protobuf-6.31.1:
  Successfully uninstalled protobuf-6.31.1
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.72.1 requires protobuf<7.0.0,>=6.30.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.72.1 requires protobuf<7.0.0,>=6.30.0, but you have protobuf 5.29.5 which is incompatible.
Note: you may need to restart

In [9]:
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

RecursiveCharacterTextSplitter est une classe de LangChain.
chunk_size=7500 signifie : chaque morceau (chunk) fera au maximum 7500 caractères.
chunk_overlap=100 signifie : chaque chunk chevauche le précédent de 100 caractères.

In [10]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [11]:
chunks = [chunk for chunk in chunks if chunk.page_content.strip() != ""]    
chunks

[Document(metadata={'source': '/home/perfect/Documents/GitHub/ollama_pdf_rag/data/pdfs/sample/1742719496377.pdf'}, page_content='Data\tScience\tfrom\tScratch\n\nThe\t#1\tData\tScience\tGuide\tfor\tEverything A\tData\tScientist\tNeeds\tto\tKnow:\tPython, Linear\tAlgebra,\tStatistics,\tCoding, Applications,\tNeural\tNetworks,\tand Decision\tTrees\n\nSteven\tCooper\n\nTable\tof\tContents\n\nPreface\n\nIntroduction\n\nData\tScience\tand\tits\tImportance\n\nWhat\tis\tit\tExactly?\n\nWhy\tIt\tMatters\n\nWhat\tYou\tNeed\n\nThe\tAdvantages\tto\tData\tScience Data\tScience\tand\tBig\tData\n\nKey\tDifference\tBetween\tData\tScience\tand\tBig\tData\n\nData\tScientists\n\nThe\tProcess\tof\tData\tScience Responsibilities\tof\ta\tData\tScientist Qualifications\tof\tData\tScientists Would\tYou\tBe\ta\tGood\tData\tScientist?\n\nThe\tImportance\tof\tHacking The\tImportance\tof\tCoding\n\nWriting\tProduction-Level\tCode\n\nPython SQL\n\nR\n\nSAS\n\nJava Scala\n\nJulia\n\nHow\tto\tWork\twith\tData\n\nDat

In [12]:
# 3. Set the environment variable
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

# 4. Now reimport with the new versions
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

# 5. Try creating the vector database
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    persist_directory="./chroma_db",
    collection_name="local-rag"
)



AttributeError: `np.float_` was removed in the NumPy 2.0 release. Use `np.float64` instead.

In [ ]:
vector_db.persist() 

In [ ]:
from chromadb import PersistentClient

client = PersistentClient(path="./chroma")  # Mets le bon chemin ici si tu as utilisé `persist_directory=...`

# Liste toutes les collections existantes
collections = client.list_collections()
print("Collections existantes :", [c.name for c in collections])


Collections existantes : []


## Retrieval

In [ ]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [ ]:
# LLM from Ollama
local_model = "hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF"  # or "llama3:8b" for the 8B version
llm = ChatOllama(model=local_model)

In [ ]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

vector_db.as_retriever() : transforme la base Chroma (vector_db) en retriever, c'est-à-dire un objet capable de retrouver des documents pertinents via similarité sémantique.

llm : c’est un modèle de langage comme ChatOpenAI() ou Ollama(). Il est utilisé pour générer plusieurs reformulations de la question.

QUERY_PROMPT : un template qui dit au LLM comment reformuler une requête utilisateur pour améliorer la recherche.

Le template est une chaîne de prompt structurée : elle demande au modèle de répondre uniquement à partir du contexte récupéré.

ChatPromptTemplate.from_template(template) crée un prompt structuré que LangChain peut remplir automatiquement avec :

{context} → les documents retrouvés

{question} → la question utilisateur

In [ ]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

Ce code crée une chaîne RAG qui utilise un retriever pour extraire du contexte pertinent à partir d'une question, puis génère une réponse avec un modèle de langage (LLM). Le résultat final est formaté en texte brut grâce à `StrOutputParser()`.

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke("The main goal of this book is ")

"The main goal of the book is not explicitly stated in the provided text snippet. However, based on the content and structure of the passage, it appears to be a tutorial or textbook introduction to vectors and vector operations.\n\nThe first part of the passage introduces the concept of vectors and provides an explanation of how they work. The second section explains the basic operations that can be performed with vectors, such as addition, subtraction, scalar multiplication, dot product, and cross product.\n\nIf I had to make a guess, I would say that the main goal of this book is likely to introduce readers to the world of vectors and their applications in various fields, such as physics, engineering, and computer science. The passage suggests that the book will cover a range of topics related to vectors, including scalar operations, vector addition, subtraction, and other advanced concepts.\n\nIf you're looking for a specific goal or outcome from this book, it would be helpful to kn

In [ ]:
# Delete all collections in the db
vector_db.delete_collection()